In [1]:
import json
import re
import requests
import sys
import io
import pandas as pd


## Import Neo4j

auth =(username, password)

In [2]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687",auth=("neo4j","a"))
session=driver.session()

## Image Data Resource

    This section uses the input file "screenid" that contains a list of screen IDs to get annotations from https://idr.openmicroscopy.org/

In [3]:
key1 = [] # contains the key
value1 = [] # contains the value
total1 = [] # holds the output


def idr(inputfile):
    IDR_BASE_URL = "https://idr.openmicroscopy.org"
    INDEX_PAGE = "%s/webclient/?experimenter=-1" % IDR_BASE_URL
    # create http session
    with requests.Session() as session:
        request = requests.Request('GET', INDEX_PAGE)
        prepped = session.prepare_request(request)
        response = session.send(prepped)
        if response.status_code != 200:
            response.raise_for_status()

    MAP_URL = "{base}/webclient/api/annotations/?type=map&{type}={screen_id}"

    with io.open(inputfile, 'r', encoding="utf-8") as file_input:
        for i in file_input.readlines():
            qs = {'base': IDR_BASE_URL, 'type': 'screen', 'screen_id': i}
            url = MAP_URL.format(**qs)
            for a in session.get(url).json()['annotations']:
                #print("****************************************************")
                namespace = a['ns']
                for v in a['values']:
                    key = v[0]
                    value = v[1]
                    r = re.sub(r'http\S+$', "", value)
                    a=r.strip()
                    key1.append(key)
                    value1.append(a)
            x ={k: b for k, b in zip(key1, value1)} 
            
            x["Screen_Id"] = i.strip('\n')
            
            
            total1.append(x)
            
            
          
                                                       
            



                    
                    
                           
idr("ex1.pmid")

### Create dataframe from the output

In [23]:
df = pd.DataFrame.from_dict(total1)


## Pubtator 

 This section uses the input file "pubmedid" that contains PubMed IDs to get annotations from https://www.ncbi.nlm.nih.gov/research/pubtator/

In [21]:
ids = []
abstract = []
final = []
title = []
def SubmitPMIDList(Inputfile, Format, Bioconcept):
    json = {}
    #
    # load pmids
    #
    with io.open(Inputfile, 'r', encoding="utf-8") as file_input:
        json = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
        ids1 = (json["pmids"])
        
    #
    # load bioconcepts
    #
    if Bioconcept != "":
        json["concepts"] = Bioconcept.split(",")

    #
    # request
    #
    r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/" + Format, json=json)
    if r.status_code != 200:
        print("[Error]: HTTP code " + str(r.status_code))
    else:
        full_text=r.text
        #print(full_text)
        final.append(full_text)
        

        # Writing to sample.json
        with open("output.json", "w") as outfile:
            
            outfile.write(full_text)
            print("done")
            
SubmitPMIDList("ex2.pmid", "biocjson","dieases")

[Error]: HTTP code 502


In [6]:
List1 = []
with open('sample142.json') as f:
    for jsonObj in f:
        PubTatorDict = json.loads(jsonObj)
        List1.append(PubTatorDict)

## Neo4j

In [17]:
for j in List1: #pubtator
    for i in total1: #idr
        session.run("""
        MERGE (a:id {name:$name})
        MERGE (b:organism {name:$name1})
        MERGE (c:pubmedID {name:$name2})
        MERGE (d:Screen_Technology_Type {name:$name3})
        MERGE (e:Study_Type {name:$name4})
        MERGE (f:PMCID {name:$name5})
        MERGE (g:Screen_Type {name:$name6})
        MERGE (h:Imaging_Method {name:$name7})
        MERGE (i:pubid {name:$name8})
        MERGE (j:year {name:$name9})
        MERGE (k:author {name:$name10})
        MERGE (l:Publication_Title {name:$name11})
        MERGE (m:Publication_Author {name:$name12})
        MERGE (n:Publication_DOI {name:$name13})
        
        
        MERGE (a)-[:isOrganism]->(b) 
        MERGE (a)-[:isID]->(c) 
        MERGE (a)-[:isScreenType]->(d)
        MERGE (a)-[:Type]->(e) 
        MERGE (a)-[:Type]->(f) 
        MERGE (a)-[:Type]->(g)
        MERGE (a)-[:Type]->(h)
        MERGE (a)-[:Type]->(l) 
        MERGE (a)-[:Type]->(m)
        MERGE (a)-[:Type]->(n)
        MERGE (i)-[:Type]->(j)
        MERGE (i)-[:isAuthor]->(k)
        
        """ ,name= i["Screen_Id"],name1= i["Organism"],name2= i["PubMed ID"], 
                    name3 =i["Screen Technology Type"], name4=i["Study Type"], 
                    name5=i["PMC ID"], name6=i["Screen Type"], name7=i["Imaging Method"],
                    name8 = j["id"], name9= j["year"], name10 = j["authors"] ,name11= i["Publication Title"]
                    ,name12= i["Publication Authors"],name13= i["Publication DOI"])

## Merge the data from pubtator and idr

In [20]:
session.run("""match(n:pubmedID) match(p:pubid)
        where n.name=p.name
        create(n)-[:connected_graph]->(p)
        """)

## Link Prediction

### This cypher queries are run in the Neo4j browser

## Common Neighbors

match(n:pubmedID) match(p:pubid)

RETURN gds.alpha.linkprediction.commonNeighbors(n, p) AS score

## Adamic Adar

match(n:pubmedID) match(p:pubid)

RETURN gds.alpha.linkprediction.adamicAdar(n, p) AS score

## Total Neighbor

match(n:pubmedID) match(p:pubid)

RETURN gds.alpha.linkprediction.totalNeighbors(n, p) AS score

## Same Community

match(n:pubmedID) match(p:pubid)

RETURN gds.alpha.linkprediction.sameCommunity(n, p) AS score